Registro de los mejores makespans y número de iteraciones

## Best Move

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load data to dataframes
def load_data(data_directory, name):
    dfs = []

    for i in range(number_of_files):
        dfs.append([])
        file_df = pd.read_csv(data_directory + name.replace("N", str(i + 1)))
        column_names = file_df.columns.values.tolist()

        # remove trailing spances from column names
        column_names = [column_name.strip() for column_name in column_names]
        file_df.columns = column_names

        idx = [-1] + file_df[file_df['current_makespan'] == ' current_makespan'].index.tolist()

        for j in range(number_of_examples - 1):
            dfs[i].append(file_df.loc[idx[j] + 1:idx[j + 1] - 2,:])

        dfs[i].append(file_df.loc[idx[number_of_examples - 1] + 1:,:])
        
    return dfs

In [6]:
numeric_columns = ['best_makespan', 'current_makespan', 'current_temp', 'iter_number_in_temp',
                      'iter_per_temp', 'max_temperature', 'min_temperature', 'num_jobs',
                      'num_machines', 'num_operations', 'temp_change_ratio', 'worst_makespan', 'prob_acceptance']

string_columns = ['should_accept', 'move_direction', 'equal_count','first_op_moved', 'last_op_moved', 
                      'longer_count', 'operation_duration', 'selected_machine', 'shorter_count', 
                      'switch_operation_1', 'violates_constraint','moves']

def cast_types(dfs, numeric_columns, string_columns):
    # for each dataframe remove the first row from data
    for file_df in dfs:
        for df in file_df:
            df.drop(df.index[0], inplace=True)

    # for each dataframe, cast columns to appropriate types
    for file in range(number_of_files):
        for example in range(number_of_examples):
            for column in numeric_columns:
                if column in dfs[file][example].columns.values.tolist():
                    dfs[file][example].loc[:, column] = \
                            dfs[file][example].loc[:,column].apply(pd.to_numeric, errors='coerce')

            for column in string_columns:
                if column in dfs[file][example].columns.values.tolist():
                    dfs[file][example].loc[:, column] = \
                            dfs[file][example].loc[:,column].astype(str)

In [36]:
def highlight_min(s):
    '''
    highlight the minimum in a Series yellow.
    '''
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [5]:
heuristics = ["BEST", "BEST2", "FIRST", "SEQ", "DE", "PS"]
files = ["JSSP_3_fN.csv", "JSSP_4_fN.csv", "JSSP_5_fN.csv", "JSSP_6_fN.csv", "JSSP_7_fN.csv", "JSSP_2_fN.csv"]
directory = "../../data/first_run/"
number_of_files = 7
number_of_examples = 20
all_dfs = {}
makespans_dfs = {}
iterations_dfs = {}

for heuristic, file in zip(heuristics, files):
    all_dfs[heuristic] = load_data(directory, file)
    cast_types(all_dfs[heuristic], numeric_columns, string_columns)

    makespans_dfs[heuristic] = pd.DataFrame()
    iterations_dfs[heuristic] = pd.DataFrame()
    for file in range(number_of_files):
        values = []
        iters = []
        for example in range(10):
            values.append(all_dfs[heuristic][file][example].loc[:, "best_makespan"].min())
            iters.append(all_dfs[heuristic][file][example].shape[0])
            
        column_name = "{0}x{1}:".format(int(all_dfs[heuristic][file][0].loc[0:1, "num_jobs"]), 
                                        int(all_dfs[heuristic][file][0].loc[0:1, "num_machines"]))
        
        makespans_dfs[heuristic][column_name] = pd.Series(values)
        iterations_dfs[heuristic][column_name] = pd.Series(iters)

In [8]:
from IPython.display import display

for heuristic in heuristics:
    print(heuristic)
    display(makespans_dfs[heuristic].style.apply(highlight_min))

BEST


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2031,2736,3412,3601,4256,5141,6417
1,2305,2640,3140,3761,4412,5097,6091
2,2465,2820,3323,3489,4500,4931,6230
3,2061,2586,2832,3235,4187,5672,6160
4,2497,2509,3152,3769,4233,5305,6409
5,2260,2662,2904,3305,4184,5231,6192
6,2262,2618,3466,3841,4801,5328,5846
7,2320,2653,3273,3708,4526,5193,5956
8,2120,2655,2780,3300,4398,5361,6338
9,2369,2720,3334,3847,3897,4712,6303


BEST2


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2073,2567,2946,3213,4031,5192,5474
1,2114,2395,2745,3581,4232,5019,6021
2,1994,2467,3552,3032,3697,5060,6387
3,2161,2618,2945,3486,3819,5369,6121
4,2140,2531,2977,3455,4059,4982,5620
5,2241,2728,2940,3390,4032,4809,6039
6,2061,2545,3045,3157,4075,5117,6144
7,2221,2667,3178,3209,3727,4938,5754
8,2284,2810,3613,3353,4130,5503,6014
9,2113,2702,2868,3687,4930,5187,6087


FIRST


...

,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2294,2277,2847,3382,4133,5052,6002
1,2209,2223,3068,3500,4249,5020,5411
2,2135,2311,3057,3416,3866,5083,6033
3,2382,2487,2675,3060,3883,4723,5845
4,2041,2323,2902,3098,4082,5029,5687
5,1920,2556,2718,3340,4108,4904,5039
6,1966,2473,2742,3345,3870,4925,5583
7,2129,2684,2923,3475,4144,5283,5556
8,2182,2312,2927,3582,4196,5338,5525
9,2016,2561,3009,2987,3757,4918,5450


DE


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2130,2785,3065,3432,4340,5048,5680
1,2221,2498,3236,2810,4011,4626,5241
2,2386,2547,2863,3207,4221,5136,5359
3,2118,2362,2939,3440,3830,4984,5171
4,2083,2477,3001,3294,4138,5356,5903
5,2192,2487,3052,3175,4243,4701,5496
6,2076,2810,3078,3310,4368,4751,5641
7,2151,2138,3365,3106,4288,5200,5999
8,2173,2460,2912,3489,3921,4567,5991
9,2362,2465,3283,3479,3756,4852,5779


PS


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2210,2678,3070,3258,4528,5306,5687
1,2143,2640,3155,3424,4291,5459,5908
2,2172,2406,3185,3573,3866,4755,5796
3,2366,2690,3378,3276,4413,5137,6386
4,2459,2406,3625,3628,4389,5158,5466
5,2199,2761,3244,3299,4204,5220,5719
6,2216,2379,3261,3344,4478,5232,6100
7,2504,2492,3415,3221,4034,4861,5972
8,2301,2234,3139,3406,4616,5221,5721
9,2419,2665,3308,3199,4037,5336,5774


In [9]:
for heuristic in heuristics:
    print(heuristic)
    display(iterations_dfs[heuristic].style.apply(highlight_max))

BEST


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,271,271,271,271,271,271,271
1,271,271,271,271,271,271,271
2,271,271,271,271,271,271,271
3,271,271,271,271,271,271,271
4,271,271,271,271,271,271,271
5,271,271,271,271,271,271,271
6,271,271,271,271,271,271,271
7,271,271,271,271,271,271,271
8,271,271,271,271,271,271,271
9,271,271,271,271,271,271,271


BEST2


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,290,285,281,291,291,296,290
1,284,295,285,280,289,285,282
2,289,295,282,285,283,281,284
3,291,293,287,292,282,285,285
4,286,285,289,289,284,285,288
5,280,293,284,287,284,285,286
6,282,288,283,290,288,287,277
7,294,284,284,294,286,288,284
8,288,290,284,284,283,286,287
9,289,289,280,283,291,285,281


FIRST


...

,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,74,58,60,43,40,30,31
1,74,61,56,41,48,32,30
2,68,59,62,53,43,24,30
3,71,58,58,46,41,32,31
4,79,64,65,45,43,30,28
5,76,54,64,45,41,28,32
6,70,57,54,46,46,30,30
7,73,60,52,41,38,33,32
8,71,62,60,46,39,32,34
9,82,56,54,39,41,30,32


DE


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,799,799,799,799,799,799,799
1,799,799,799,799,799,799,799
2,799,799,799,799,799,799,799
3,799,799,799,799,799,799,799
4,799,799,799,799,799,799,799
5,799,799,799,799,799,799,799
6,799,799,799,799,799,799,799
7,799,799,799,799,799,799,799
8,799,799,799,799,799,799,799
9,799,799,799,799,799,799,799


PS


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,799,799,799,799,799,799,799
1,799,799,799,799,799,799,799
2,799,799,799,799,799,799,799
3,799,799,799,799,799,799,799
4,799,799,799,799,799,799,799
5,799,799,799,799,799,799,799
6,799,799,799,799,799,799,799
7,799,799,799,799,799,799,799
8,799,799,799,799,799,799,799
9,799,799,799,799,799,799,799


## Algoritmos basados en intuición

Con las observaciones realizadas y las conclusiones derivadas a partir de ellas, se determinó la existencia de mejora considerable al realizar una hiperheuristica que tome las ventajas de las heuristicas utilizadas solamente. Por ello, se esarrollaron los siguientes algoritmos:

### PS Exchange with ascending sequential search interleaved with best move algorithm

In general, PS Exchange followed by P and S Exchange will be given 100 iterations without affecting temperature at the beginning, its main function is to find a suitable initial solution for the rest of the heuristics. The reson for this selection is based on the rapid decrease in makespan done by the heuristic. This can be seen in the following graphs:

![ps_1](pictures/ps_1.png)

Once a candidate solution is obtained, best move will start being used for 70 iterations. However, if 30 iterations pas without change using the two methods above, sequential search will take over the remaining 70 iterations. After this, Best Move will begin excution under two scenarios.

1. If 10 iterations pass with best move and no improvement is obtained, change best move for sequential search.
2. Best Move's 70 iterations pass, at which sequential search will begin and do 5 machine selections. This is equivalent to $5n$ iterations in the worst case, where n is the number of operations in the machine.

The algorithm for now will run for some ~800 iterations, ~900 if we count the candidate selection. Therefore, in average it is expected that the heuristic distribution will be:

* 0   - 100: PS Exchange, P and S Exchange, Sequential (under certain circumstances)
* 100 - 170: Best Move
* 170 - 240: Sequential Search
* 240 - 310: Best Move
* 310 - 400: Sequential Search
* 400 - 450: Best Move
* 450 - 510: Sequential Search
* ... 

In [17]:
heuristics = ["H1", "H2", "H3", "H4"]
files = ["JSSPH1_fN.csv", "JSSPH2_fN.csv", "JSSPH3_fN.csv", "JSSPH4_fN.csv"]
directory = "../../data/hyperheuristics_first_run/"
number_of_files = 7
number_of_examples = 11
all_dfs = {}
makespans_dfs = {}
iterations_dfs = {}

string_columns.append("heuristic")

for heuristic, file in zip(heuristics, files):
    all_dfs[heuristic] = load_data(directory, file)
    cast_types(all_dfs[heuristic], numeric_columns, string_columns)

    makespans_dfs[heuristic] = pd.DataFrame()
    iterations_dfs[heuristic] = pd.DataFrame()
    for file in range(number_of_files):
        values = []
        iters = []
        for example in range(10):
            values.append(all_dfs[heuristic][file][example].loc[:, "best_makespan"].min())
            iters.append(all_dfs[heuristic][file][example].shape[0])
            
        column_name = "{0}x{1}:".format(int(all_dfs[heuristic][file][0].loc[0:1, "num_jobs"]), 
                                        int(all_dfs[heuristic][file][0].loc[0:1, "num_machines"]))
        
        makespans_dfs[heuristic][column_name] = pd.Series(values)
        iterations_dfs[heuristic][column_name] = pd.Series(iters)

In [12]:
from IPython.display import display

for heuristic in heuristics:
    print(heuristic)
    display(makespans_dfs[heuristic].style.apply(highlight_min))

H1


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2313,2558,3224,3521,4066,5012,5753
1,1814,2496,2733,3264,4121,5421,5677
2,2208,2497,3199,3499,3883,4989,6192
3,1847,2479,3125,3972,4290,5357,5745
4,2198,2481,3199,3489,4125,4984,5982
5,1800,2443,3319,3180,4098,5165,5993
6,2184,2631,3109,3442,4685,4906,5860
7,2016,2439,3159,3303,4226,4997,6110
8,2186,2528,3045,3279,4038,4829,5358
9,1955,2308,2929,3023,3867,4631,5556


H2


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2191,2375,3003,3216,4070,5305,5922
1,2091,2365,3288,3419,4293,5048,5820
2,2241,2344,3005,3892,4004,5056,5918
3,1915,2683,3042,3233,4206,5089,5444
4,2436,2559,3194,3170,4027,5148,6072
5,2069,2427,3262,3567,4584,5247,5686
6,1959,2317,3220,3367,4070,5719,5949
7,2280,2628,3047,3357,4458,4919,5901
8,1919,2163,3103,3453,4156,4989,5874
9,1844,2283,2812,2927,3747,4697,5422


H3


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,1969,2747,2930,3347,4571,4835,5989
1,2852,2264,2978,3474,4530,4877,5740
2,1974,2532,3072,3806,4181,5373,5860
3,2050,2323,3179,3593,4383,4830,5536
4,2359,2546,2954,3396,4194,4726,5706
5,2097,2446,2829,3387,4027,4630,5736
6,2046,2570,3175,3370,4188,5032,5976
7,2122,2416,2829,3323,3730,5109,5402
8,1968,2405,2877,3530,3786,4905,6033
9,1903,2281,2781,3137,3874,4576,5404


H4


,15x15:,20x15:,20x20:,30x15:,30x20:,50x15:,50x20:
0,2284,2446,2787,3523,4345,5238,5883
1,2053,2474,3030,3234,4532,5420,6013
2,2149,2514,3024,3217,4065,5386,6066
3,2429,2170,2966,3186,4228,5010,5507
4,2076,2687,2999,3492,4331,5383,5819
5,2127,2360,2962,3351,3955,5062,5585
6,2050,2638,2902,3098,3965,5093,5828
7,2018,2522,3029,3143,4116,5031,5713
8,1903,2259,2968,3542,4156,5189,6099
9,1914,2113,2783,3116,3600,4690,5542


In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bokeh.plotting import figure, show
from bokeh.io import gridplot, output_notebook
from bokeh.charts import Bar
import bokeh.palettes

# load plotting library
output_notebook()

Loading BokehJS ...

In [18]:
# the best makespan found till that moment per iteration
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(10):
        ys.append(all_dfs["H1"][file][example].loc[:, "best_makespan"].tolist())
        xs.append(list(range(all_dfs["H1"][file][example].shape[0])))
    
    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][10], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Best Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Best Makespan per Iteration Comparison")
show(p)

In [19]:
# the best makespan found till that moment per iteration
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(10):
        ys.append(all_dfs["H2"][file][example].loc[:, "best_makespan"].tolist())
        xs.appenad(list(range(all_dfs["H2"][file][example].shape[0])))
    
    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][10], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Best Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Best Makespan per Iteration Comparison")
show(p)

In [22]:
# the best makespan found till that moment per iteration
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(10):
        ys.append(all_dfs["H3"][file][example].loc[:, "best_makespan"].tolist())
        xs.append(list(range(all_dfs["H3"][file][example].shape[0])))
    
    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][10], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Best Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Best Makespan per Iteration Comparison")
show(p)

In [21]:
# the best makespan found till that moment per iteration
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(10):
        ys.append(all_dfs["H4"][file][example].loc[:, "best_makespan"].tolist())
        xs.append(list(range(all_dfs["H4"][file][example].shape[0])))
    
    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][10], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Best Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Best Makespan per Iteration Comparison")
show(p)

In [23]:
# muestra el makespan actual con respecto a las iteraciones realizadas
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(5):
        ys.append(all_dfs["H1"][file][example].loc[:, "current_makespan"].tolist())
        xs.append(list(range(all_dfs["H1"][file][example].shape[0])))

    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][5], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Current Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Current Makespan per Iteration Comparison")
show(p)

In [24]:
# muestra el makespan actual con respecto a las iteraciones realizadas
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(5):
        ys.append(all_dfs["H2"][file][example].loc[:, "current_makespan"].tolist())
        xs.append(list(range(all_dfs["H2"][file][example].shape[0])))

    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][5], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Current Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Current Makespan per Iteration Comparison")
show(p)

In [25]:
# muestra el makespan actual con respecto a las iteraciones realizadas
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(5):
        ys.append(all_dfs["H3"][file][example].loc[:, "current_makespan"].tolist())
        xs.append(list(range(all_dfs["H3"][file][example].shape[0])))

    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][5], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Current Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Current Makespan per Iteration Comparison")
show(p)

In [26]:
# muestra el makespan actual con respecto a las iteraciones realizadas
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(5):
        ys.append(all_dfs["H4"][file][example].loc[:, "current_makespan"].tolist())
        xs.append(list(range(all_dfs["H4"][file][example].shape[0])))

    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][5], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Current Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Current Makespan per Iteration Comparison")
show(p)

In [28]:
# the best makespan found till that moment per iteration
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []
    colors = []
    
    example = 0
    
    heuristics = set(all_dfs["H1"][file][example].loc[:, "heuristic"].tolist())
    df = all_dfs["H1"][file][example].loc[:, ["best_makespan", "heuristic"]]
    df["x_axis"] = list(range(all_dfs["H1"][file][example].shape[0]))
    
    for heuristic in heuristics:
        h_df = df[df["heuristic"] == heuristic]
        all_ys = h_df.loc[:, "best_makespan"].tolist()
        all_xs = h_df.loc[:, "x_axis"].tolist()

        start = 0
        for i in range(1, h_df.shape[0]) :
            if abs(all_xs[i-1] - all_xs[i]) > 1:   
                ys.append(all_ys[start:i])
                xs.append(all_xs[start:i])
                start = i
        
                if heuristic == " SEQUENTIAL":
                    colors.append("#13719f")
                elif heuristic == " BEST":
                    colors.append("blue")
                elif heuristic == " P_ANS_S":
                    colors.append("green")
                else:
                    colors.append("red")
                    
        ys.append(all_ys[start:i])
        xs.append(all_xs[start:i])
        start = i

        if heuristic == " SEQUENTIAL":
            colors.append("orange")
        elif heuristic == " BEST":
            colors.append("blue")
        elif heuristic == " P_AND_S":
            colors.append("green")
        else:
            colors.append("red")
    
    p.multi_line(xs, ys, color=colors, line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Best Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Best Makespan per Iteration Comparison")
show(p)

In [35]:
all_dfs["H1"][0][0].dtypes

best_makespan            int64
current_makespan         int64
current_temp           float64
equal_count             object
first_op_moved          object
heuristic               object
iter_number_in_temp      int64
iter_per_temp            int64
last_op_moved           object
longer_count            object
max_temperature        float64
min_temperature        float64
move_direction          object
moves                   object
num_jobs                 int64
num_machines             int64
num_operations         float64
operation_duration      object
prob_acceptance        float64
selected_machine        object
shorter_count           object
should_accept           object
switch_operation_1      object
temp_change_ratio      float64
times_moved             object
violates_constraint     object
worst_makespan           int64
                        object
dtype: object

## H2

This hyper-heuristic is a variation of the first in which sequential search is replaced by reverse sequential search due to the intuition that doing it from the last operations has a greater chance of success. This was derived from the fact that most of the time, it takes sequential search around the same number of swaps as operations in a mahchine to find a solution.

## H3

Is the same algoritm as H1, but it does the heuristics in reverse order, that means that when the initial 100 iterations are over, sequential search takes over instead of best move. After such change, the algorithm behaves the same.

## H4

Variation of H3 that uses reverse sequential search instead of sequential search.

In [87]:
heuristics = ["H1", "H2", "H3", "H4"]
files = ["JSSPH1_100_fN.csv", "JSSPH2_100_fN.csv", "JSSPH3_100_fN.csv", "JSSPH4_100_fN.csv"]
directory = "../../data/hyperheuristics_second_run/"
number_of_files = 1
number_of_examples = 11
all_dfs = {}
makespans_dfs = {}
iterations_dfs = {}

string_columns.append("heuristic")

for heuristic, file in zip(heuristics, files):
    all_dfs[heuristic] = load_data(directory, file)
    cast_types(all_dfs[heuristic], numeric_columns, string_columns)

    makespans_dfs[heuristic] = pd.DataFrame()
    iterations_dfs[heuristic] = pd.DataFrame()
    for file in range(number_of_files):
        values = []
        iters = []
        for example in range(10):
            values.append(all_dfs[heuristic][file][example].loc[:, "best_makespan"].min())
            iters.append(all_dfs[heuristic][file][example].shape[0])
            
        column_name = "{0}x{1}:".format(int(all_dfs[heuristic][file][0].loc[0:1, "num_jobs"]), 
                                        int(all_dfs[heuristic][file][0].loc[0:1, "num_machines"]))
        
        makespans_dfs[heuristic][column_name] = pd.Series(values)
        iterations_dfs[heuristic][column_name] = pd.Series(iters)

In [88]:
from IPython.display import display

for heuristic in heuristics:
    print(heuristic)
    display(makespans_dfs[heuristic].style.apply(highlight_min))

H1


,15x15:
0,2601
1,3623
2,2616
3,4068
4,2463
5,2643
6,3764
7,3621
8,2568
9,3860


H2


,15x15:
0,2663
1,2560
2,3592
3,2479
4,2637
5,3388
6,4131
7,2349
8,2697
9,2714


H3


,15x15:
0,3670
1,3937
2,3540
3,2488
4,2513
5,2488
6,2617
7,2565
8,3457
9,2560


H4


,15x15:
0,3524
1,2448
2,2356
3,3721
4,2557
5,2551
6,3887
7,2608
8,4026
9,2560


In [89]:
# the best makespan found till that moment per iteration
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(10):
        counter = 0
        y = []
        x = []
        previous = 1
        
        df = all_dfs["H1"][file][example].loc[:, ["best_makespan", "heuristic", "times_moved"]]
        
        for (best, heur), times in zip(zip(df.loc[:, "best_makespan"].tolist(),df.loc[:, "heuristic"].tolist()), 
                                       df.loc[:, "times_moved"].tolist()):
            index = 1
            if heur == " BEST":
                index = 3
            elif heur == " SEQUENTIAL" or heur == " REVERSE_SEQUENTIAL":
                num = times.strip()
                
                if num != "":
                    index = int(num)
                else:
                    index = int(previous)
                    
            previous = times
                
            for i in range(index):
                x.append(counter)
                counter += 1
                y.append(best)
        
        ys.append(y)
        xs.append(x)
    
    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][10], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Best Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Best Makespan per Iteration Comparison")
show(p)

In [82]:
heuristics = ["H1"]
files = ["JSSPH1_100_2_fN.csv"]
directory = "../../data/hyperheuristics_second_run/"
number_of_files = 1
number_of_examples = 11
all_dfs = {}
makespans_dfs = {}
iterations_dfs = {}

string_columns.append("heuristic")

for heuristic, file in zip(heuristics, files):
    all_dfs[heuristic] = load_data(directory, file)
    cast_types(all_dfs[heuristic], numeric_columns, string_columns)

    makespans_dfs[heuristic] = pd.DataFrame()
    iterations_dfs[heuristic] = pd.DataFrame()
    for file in range(number_of_files):
        values = []
        iters = []
        for example in range(10):
            values.append(all_dfs[heuristic][file][example].loc[:, "best_makespan"].min())
            iters.append(all_dfs[heuristic][file][example].shape[0])
            
        column_name = "{0}x{1}:".format(int(all_dfs[heuristic][file][0].loc[0:1, "num_jobs"]), 
                                        int(all_dfs[heuristic][file][0].loc[0:1, "num_machines"]))
        
        makespans_dfs[heuristic][column_name] = pd.Series(values)
        iterations_dfs[heuristic][column_name] = pd.Series(iters)

In [83]:
from IPython.display import display

for heuristic in heuristics:
    print(heuristic)
    display(makespans_dfs[heuristic].style.apply(highlight_min))

H1


,15x15:
0,2857
1,2967
2,3695
3,2863
4,3816
5,2719
6,2870
7,3426
8,2837
9,2833


In [86]:
# the best makespan found till that moment per iteration
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(10):
        counter = 0
        y = []
        x = []
        previous = 1
        
        df = all_dfs["H1"][file][example].loc[:, ["best_makespan", "heuristic", "times_moved"]]
        
        for (best, heur), times in zip(zip(df.loc[:, "best_makespan"].tolist(),df.loc[:, "heuristic"].tolist()), 
                                       df.loc[:, "times_moved"].tolist()):
            index = 1
            if heur == " BEST":
                index = 3
            elif heur == " SEQUENTIAL" or heur == " REVERSE_SEQUENTIAL":
                num = times.strip()
                
                if num != "":
                    index = int(num)
                else:
                    index = int(previous)
                    
            previous = times
                
            for i in range(index):
                x.append(counter)
                counter += 1
                y.append(best)
        
        ys.append(y)
        xs.append(x)
    
    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][10], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Best Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Best Makespan per Iteration Comparison")
show(p)

In [90]:
heuristics = ["H1"]
files = ["JSSPH1_100_2aux_fN.csv"]
directory = "../../data/hyperheuristics_second_run/"
number_of_files = 1
number_of_examples = 11
all_dfs = {}
makespans_dfs = {}
iterations_dfs = {}

string_columns.append("heuristic")

for heuristic, file in zip(heuristics, files):
    all_dfs[heuristic] = load_data(directory, file)
    cast_types(all_dfs[heuristic], numeric_columns, string_columns)

    makespans_dfs[heuristic] = pd.DataFrame()
    iterations_dfs[heuristic] = pd.DataFrame()
    for file in range(number_of_files):
        values = []
        iters = []
        for example in range(10):
            values.append(all_dfs[heuristic][file][example].loc[:, "best_makespan"].min())
            iters.append(all_dfs[heuristic][file][example].shape[0])
            
        column_name = "{0}x{1}:".format(int(all_dfs[heuristic][file][0].loc[0:1, "num_jobs"]), 
                                        int(all_dfs[heuristic][file][0].loc[0:1, "num_machines"]))
        
        makespans_dfs[heuristic][column_name] = pd.Series(values)
        iterations_dfs[heuristic][column_name] = pd.Series(iters)

In [91]:
from IPython.display import display

for heuristic in heuristics:
    print(heuristic)
    display(makespans_dfs[heuristic].style.apply(highlight_min))

H1


,15x15:
0,2711
1,2831
2,3017
3,2794
4,3048
5,2751
6,2871
7,2884
8,2692
9,3158


In [92]:
# the best makespan found till that moment per iteration
plots = []
for file in range(number_of_files):
    p = figure(plot_width=4000, plot_height=400)
    ys = []
    xs = []

    for example in range(10):
        counter = 0
        y = []
        x = []
        previous = 1
        
        df = all_dfs["H1"][file][example].loc[:, ["best_makespan", "heuristic", "times_moved"]]
        
        for (best, heur), times in zip(zip(df.loc[:, "best_makespan"].tolist(),df.loc[:, "heuristic"].tolist()), 
                                       df.loc[:, "times_moved"].tolist()):
            index = 1
            if heur == " BEST":
                index = 3
            elif heur == " SEQUENTIAL" or heur == " REVERSE_SEQUENTIAL":
                num = times.strip()
                
                if num != "":
                    index = int(num)
                else:
                    index = int(previous)
                    
            previous = times
                
            for i in range(index):
                x.append(counter)
                counter += 1
                y.append(best)
        
        ys.append(y)
        xs.append(x)
    
    p.multi_line(xs, ys, color=bokeh.palettes.d3['Category10'][10], line_width=2)
    
    p.xaxis.axis_label = "Iteration"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Best Makespan"
    p.yaxis.axis_label_text_color = "#aa6666"
    
    plots.append([p])

p = gridplot(plots, toolbar_location="left", title="Best Makespan per Iteration Comparison")
show(p)